In [38]:
import pandas as pd
import nltk
from cleantext import clean


In [ ]:
df = pd.read_csv('data/news_sample.csv')

In [43]:
def clean_text_library(text):
    return clean(text,
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=True,            # fully strip line breaks as opposed to only normalizing them
    lang="en"                       
)

# Cleaning the content column of the dataset
df['content'] = df['content'].apply(clean_text_library)
df.to_csv('data/news_sample_1.csv', index=False)

In [ ]:
# Regex patterns 
patterns = {
    r"[\w]+ [\d]+, [\d]+": "<DATE>",                    # DATE method 1
    r"[\d]+[\w]+ [\w]+ [\d]+": "<DATE>",                # DATE method 2
    r"[\d]+\/?-?.?[\d]+\/?-?.?[[\d]+":"<DATE>",         # DATE method 3
    r"[\w]+ \d\d?[\w]?[\w]?,? [\d]{2,4}":"<DATE>",      # DATE method 4
    r"([\d]{,2}[\w]*) ([\w]*),? ([\d]{2,4})":"<DATE>",  # DATE method 5
    r"\b(\d+(st|nd|rd|th|s))\b":"<NUM>"                 # NUM
}

def clean_dates(df):
    for pattern, replacement in patterns.items():
        df["content"].str.replace(pattern, replacement, regex=True)
    return df

df = clean_dates(df)
df.to_csv('data/news_sample_2.csv', index=False)

KeyError: 'content'

In [42]:
# Using the package clean text to clean the data in the same way as I did manually 
def clean_text_library(text):
    return clean(text,
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=True,      # replace all currency symbols with a special token
    no_punct=True,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    replace_with_number="<NUMBER>",
    replace_with_digit="0",
    replace_with_currency_symbol="<CUR>",
    lang="en"                       
)

# Cleaning the content column of the dataset
df['content'] = df['content'].apply(clean_text_library)
df.to_csv('data/news_sample_3.csv', index=False)

: 

All lower
punktuation 
mails 
numbers 
